# Clustering

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.special as sp
import pycodamath as coda
from pycodamath import plot


import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import scipy.optimize as optimization

#### 1. Data Loading

In [51]:
import pandas as pd
import re

# Load data
data = pd.read_csv('../data/2-meteorites.csv', sep=',')
data.rename(columns={'Unnamed: 0': 'Location'}, inplace=True)

# Strip leading and trailing spaces from all column names
data.columns = data.columns.str.strip()

# Extract the part inside parentheses into 'Type'
data['Type'] = data['Location'].str.extract(r'\((.*?)\)')

# Remove the parentheses and content from 'Location'
data['Location'] = data['Location'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()

# Reorder columns: Location, Type, then the rest
cols = ['Location', 'Type'] + [col for col in data.columns if col not in ['Location', 'Type']]
data = data[cols]
data.iloc[:, 2:] = data.iloc[:, 2:].coda.closure(100)

# Check result
print(data.head())
print(data.shape)


    Location Type       SiO2     Al2O3        FeO       MnO        MgO  \
0    Allende   cc  37.915374  3.622065  30.073106  0.199380  27.270713   
1       Bali   cc  37.892729  3.545781  29.746409  0.213196  27.535907   
2  Efremovka   cc  39.220393  3.852309  19.661637  0.217193  28.246456   
3   Coolidge   cc  42.960559  4.131303   9.061408  0.299551  31.190714   
4    Ankober   hc  39.378411  2.584382  14.537150  0.356467  25.465077   

          Fe        Ni        Co         C  
0   0.188303  0.398759  0.011077  0.321223  
1   0.168312  0.246858  0.011221  0.639587  
2   6.298583  1.566072  0.068587  0.868770  
3  10.334498  1.684973  0.099850  0.237144  
4  15.550852  1.904868  0.111396  0.111396  
(12, 11)


In [54]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

# --- Step 1: Define your full response (all 9 parts) ---
response = data[['SiO2', 'Al2O3', 'FeO', 'MnO', 'MgO', 'Fe', 'Ni', 'Co', 'C']]
covariates = data[['Type']]  # cc, hc, lc



# --- Step 4: Perform ILR transform ---
ilr = response.coda.ilr()
ilr = ilr.rename(columns={i: f'ilr{i}' for i in range(8)})
ilr['Type'] = covariates['Type']

# --- Step 5: Run ANOVA per balance ---
print('\n--- ANOVA per ILR balance ---')
results = []
for part in ilr.columns[:-1]:
    model = ols(f'{part} ~ Type', data=ilr).fit()
    intercept = model.params.get('Intercept', np.nan)
    beta_hc = model.params.get('Type[T.hc]', np.nan)
    beta_lc = model.params.get('Type[T.lc]', np.nan)
    t_value = np.sqrt(model.fvalue)
    p_value = model.f_pvalue

    results.append({
        'balance': part,
        'intercept': intercept,
        'beta_hc_vs_cc': beta_hc,
        'beta_lc_vs_cc': beta_lc,
        't_value': t_value,
        'p_value': p_value
    })

# Print results
for res in results:
    print(f"{res['balance']}")
    print(f"Intercept (cc): {res['intercept']:.4f}")
    print(f"Beta hc vs cc: {res['beta_hc_vs_cc']:.4f}")
    print(f"Beta lc vs cc: {res['beta_lc_vs_cc']:.4f}")
    print(f"t-value: {res['t_value']:.4f}")
    print(f"p-value: {res['p_value']:.4f}")
    print()

# --- Step 6: Compute CLR back-transformed betas ---
beta_hc_arr = np.array([res['beta_hc_vs_cc'] for res in results]).reshape(1, -1)
beta_lc_arr = np.array([res['beta_lc_vs_cc'] for res in results]).reshape(1, -1)

clrbeta_hc = pd.DataFrame(np.dot(beta_hc_arr, psi), index=['clr_beta_hc'], columns=response.columns)
clrbeta_lc = pd.DataFrame(np.dot(beta_lc_arr, psi), index=['clr_beta_lc'], columns=response.columns)

print('\n--- CLR beta coefficients (hc vs cc, sorted) ---')
print(clrbeta_hc.iloc[0].sort_values())

print('\n--- CLR beta coefficients (lc vs cc, sorted) ---')
print(clrbeta_lc.iloc[0].sort_values())



--- ANOVA per ILR balance ---
ilr0
Intercept (cc): 1.4793
Beta hc vs cc: 2.1413
Beta lc vs cc: 0.5904
t-value: 2.5406
p-value: 0.0183

ilr1
Intercept (cc): 4.5676
Beta hc vs cc: -0.4926
Beta lc vs cc: -0.3511
t-value: 1.1005
p-value: 0.3421

ilr2
Intercept (cc): 1.8641
Beta hc vs cc: -0.6097
Beta lc vs cc: -0.3824
t-value: 1.7879
p-value: 0.0894

ilr3
Intercept (cc): 1.6381
Beta hc vs cc: -2.5444
Beta lc vs cc: -1.4618
t-value: 2.0286
p-value: 0.0538

ilr4
Intercept (cc): -1.5374
Beta hc vs cc: -0.0271
Beta lc vs cc: -0.0173
t-value: 0.3392
p-value: 0.8926

ilr5
Intercept (cc): 3.5846
Beta hc vs cc: -0.6473
Beta lc vs cc: -0.6437
t-value: 4.4078
p-value: 0.0005

ilr6
Intercept (cc): -0.4020
Beta hc vs cc: 0.2805
Beta lc vs cc: 0.1458
t-value: 0.8954
p-value: 0.4782

ilr7
Intercept (cc): 1.6581
Beta hc vs cc: 0.3291
Beta lc vs cc: 0.4269
t-value: 5.7758
p-value: 0.0001


--- CLR beta coefficients (hc vs cc, sorted) ---
Co      -1.213852
Ni      -1.103056
SiO2    -0.705657
Al2O3   -0.66